In [1]:
import torch 
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

%cd ../

/workspace/Walk_Video_PyTorch/project


In [2]:
from models.pytorchvideo_models import WalkVideoClassificationLightningModule
from pytorch_lightning import Trainer

from IPython.display import clear_output

clear_output()

In [3]:
from pytorch_lightning import seed_everything

seed_everything(42, workers=True)

Global seed set to 42


42

In [4]:
from main import get_parameters

opt, unknown = get_parameters()
opt.num_workers = 8
opt.batch_size = 4
opt.gpu_num = 1

opt.version = '0821_2_16'
opt.model = "resnet"
opt.model_depth = 50

opt.clip_duraion = 4
opt.uniform_temporal_subsample_num = 32
opt.version = opt.version + '_' + opt.model + '_depth' + str(opt.model_depth)


In [5]:
from utils.utils import get_ckpt_path

model = WalkVideoClassificationLightningModule(opt)

# get last ckpt path
# ckpt_path = get_ckpt_path(opt)

ckpt_path = '/workspace/Walk_Video_PyTorch/logs/resnet/0821_2_16_resnet_depth50/checkpoints/epoch=73-val_loss=0.67-val_acc=0.6119.ckpt'

# model = WalkVideoClassificationLightningModule.load_from_checkpoint(ckpt_path)
# model.load_state_dict(weight)
model = model.load_from_checkpoint(ckpt_path)

model.eval()

# clear_output()
print(ckpt_path)


/usr/local/lib/python3.9/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.9/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.9/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


/workspace/Walk_Video_PyTorch/logs/resnet/0821_2_16_resnet_depth50/checkpoints/epoch=73-val_loss=0.67-val_acc=0.6119.ckpt


In [6]:
from dataloader.data_loader import WalkDataModule
from pytorch_lightning import loggers as pl_loggers 

# load test dataset 
module = WalkDataModule(opt)
module.setup()
test_data = module.test_dataloader()

# for the tensorboard
tb_logger = pl_loggers.TensorBoardLogger(save_dir="/workspace/Walk_Video_PyTorch/project/tests/logs", name=opt.model, version=opt.version)


In [7]:

trainer = Trainer(
    accelerator="auto",
    devices=1,
    gpus=opt.gpu_num,
    logger=tb_logger,
    max_epochs=1,
    # deterministic=True,
)

trainer.test(dataloaders=module, model=model, ckpt_path=ckpt_path)


/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:460: UserWarning: The flag `devices=1` will be ignored, instead the device specific number 1 will be used
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at /workspace/Walk_Video_PyTorch/logs/resnet/0821_2_16_resnet_depth50/checkpoints/epoch=73-val_loss=0.67-val_acc=0.6119.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]
Loaded model weights from checkpoint at /workspace/Walk_Video_PyTorch/logs/resnet/0821_2_16_resnet_depth50/checkpoints/epoch=73-val_loss=0.67-val_acc=0.6119.ckpt


Testing: 0it [00:00, ?it/s]

In [ ]:
input_data = next(iter(test_data))


In [ ]:
input_data['video'].shape

input_data["video_name"]

In [ ]:
preds = model(input_data['video'])

In [ ]:
real_label = input_data['label']
pred_label = preds

real_label, pred_label


In [ ]:
from torch.nn.functional import sigmoid, tanh, softmax

sigmoid(pred_label), tanh(pred_label), softmax(pred_label)

In [ ]:
pred_label_softmax = torch.softmax(pred_label, dim =-1)
pred_label_softmax.shape, real_label.shape

In [ ]:
from torch.nn import HingeEmbeddingLoss, CrossEntropyLoss, BCEWithLogitsLoss

hingeloss = HingeEmbeddingLoss(reduction='mean')
crossloss = CrossEntropyLoss()
bceloss = BCEWithLogitsLoss()

crossloss(pred_label_softmax, real_label)

In [ ]:
preds.topk(k=1).indices, real_label

In [ ]:
from torch import softmax


pred_chagne = softmax(preds, dim=-1)
pred_chagne

In [ ]:
from utils.metrics import get_Accuracy

accuracy = get_Accuracy()

accuracy(pred_chagne, input_data['label'])

In [ ]:
classname = {}

classname[0] = 'asd'
classname[1] = 'asd_not'

In [ ]:
real_calss = []

for i in input_data['label'].tolist():
    real_calss.append(classname[i])

In [ ]:
real_calss

In [ ]:
# pred_class_names = []
# for num in range(opt._BATCH_SIZE):
#     for i in pred_classes[i]:
#         pred_class_names.append(classname[int(i)])


pred_class_names = [classname[int(i)] for i in pred_classes]
print("Predicted labels: %s" % ", ".join(pred_class_names))
print("real label: %s" % ",".join(real_calss))

In [ ]:
pred_class_names == real_calss

In [ ]:
result = []

for i in range(len(real_calss)):
    if pred_class_names[i] == real_calss[i]:
        result.append("true")
    else:
        result.append("false")

result